In [ ]:
%pip install google-generativeai pdf2image pytesseract pillow pdfplumber

In [ ]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, Optional, Tuple
from dotenv import load_dotenv
from jsonschema import validate, ValidationError
from supabase import create_client, Client

print("✓ All libraries imported")

In [ ]:
PDF_DIR = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\data"
OUTPUT_DIR = "./extracted_data"
ERROR_LOG_DIR = "./errors"

# Load environment variables
dotenv_path = r"C:\Users\samue\Documents\Work\Code\valuation_data_miner\.env"
load_dotenv(dotenv_path)

# API Keys
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Model settings
GEMINI_MODEL = "gemini-1.5-flash"  # Fast and free!
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Create directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ERROR_LOG_DIR, exist_ok=True)

print("✓ Configuration loaded")
print(f"  PDF Directory: {PDF_DIR}")
print(f"  Gemini Model: {GEMINI_MODEL}")

if not GEMINI_API_KEY:
    print("\n⚠ WARNING: GEMINI_API_KEY not found!")
    print("  Get key from: https://makersuite.google.com/app/apikey")

In [ ]:
VALUATION_SCHEMA = {
    "type": "object",
    "required": ["property_id", "valuation_report", "property_details", "valuations"],
    "properties": {
        "property_id": {"type": "string"},
        "valuation_report": {
            "type": "object",
            "properties": {
                "report_reference": {"type": "string"},
                "valuer": {"type": "string"},
                "valuers": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "qualification": {"type": "string"}
                        }
                    }
                },
                "inspection_date": {"type": "string"},
                "report_date": {"type": "string"},
                "client": {"type": "string"},
                "client_address": {"type": "string"},
                "purpose": {"type": "string"}
            }
        },
        "property_details": {
            "type": "object",
            "properties": {
                "apartment_number": {"type": "string"},
                "block": {"type": "string"},
                "floor": {"type": "string"},
                "title_details": {"type": "object"},
                "location": {"type": "object"},
                "tenure": {"type": "object"},
                "registered_proprietors": {"type": "array"},
                "ownership_type": {"type": "string"},
                "encumbrances": {"type": "string"}
            }
        },
        "property_description": {"type": "object"},
        "apartment_details": {"type": "object"},
        "occupancy": {"type": "string"},
        "condition": {"type": "string"},
        "market_assessment": {"type": "object"},
        "valuations": {
            "type": "object",
            "properties": {
                "current_market_value": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number"},
                        "currency": {"type": "string"},
                        "amount_words": {"type": "string"}
                    }
                }
            }
        },
        "valuation_methodology": {"type": "array"},
        "lease_details": {"type": "object"},
        "compliance": {"type": "object"}
    }
}

print("✓ JSON schema defined")

In [ ]:
def validate_extracted_data(data: Dict) -> Tuple[bool, Optional[str]]:
    """Validate extracted data against schema"""
    try:
        validate(instance=data, schema=VALUATION_SCHEMA)
        return True, None
    except ValidationError as e:
        return False, str(e)

print("✓ Validation function defined")


In [ ]:
def init_supabase() -> Optional[Client]:
    """Initialize Supabase client"""
    try:
        if not SUPABASE_URL or not SUPABASE_KEY:
            print("⚠ Supabase credentials not configured")
            return None
        
        client = create_client(SUPABASE_URL, SUPABASE_KEY)
        print("✓ Supabase client initialized")
        return client
    except Exception as e:
        print(f"✗ Supabase init failed: {e}")
        return None

supabase_client = init_supabase()


In [ ]:
def upload_to_supabase(client: Client, data: Dict, filename: str) -> bool:
    """Upload to Supabase"""
    if client is None:
        return False
    
    try:
        upload_data = data.copy()
        upload_data['source_filename'] = filename
        upload_data['processed_at'] = datetime.now().isoformat()
        
        client.table("property_valuations").insert(upload_data).execute()
        return True
    except Exception as e:
        raise Exception(f"Supabase upload failed: {str(e)}")

print("✓ Supabase functions defined")

In [ ]:
def process_scanned_pdf_with_ocr(pdf_path: str) -> str:
    """Convert scanned PDF to text using Tesseract OCR"""
    try:
        from pdf2image import convert_from_path
        import pytesseract
        
        # Set Tesseract path
        if os.path.exists(TESSERACT_PATH):
            pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH
        else:
            raise Exception(f"Tesseract not found at {TESSERACT_PATH}. Install from: https://github.com/UB-Mannheim/tesseract/wiki")
        
        print(f"    → Converting PDF to images...")
        
        # Convert PDF to images
        images = convert_from_path(pdf_path, dpi=300, fmt='jpeg', thread_count=2)
        
        print(f"    → Running OCR on {len(images)} pages...")
        
        # OCR each page
        all_text = []
        for i, image in enumerate(images):
            print(f"      → Page {i+1}/{len(images)}...", end=' ')
            
            text = pytesseract.image_to_string(
                image,
                lang='eng',
                config='--psm 1'
            )
            
            text = text.strip()
            
            if text:
                all_text.append(f"--- Page {i+1} ---\n{text}")
                print(f"✓ {len(text)} chars")
            else:
                print("(empty)")
        
        full_text = "\n\n".join(all_text)
        
        print(f"    → OCR complete: {len(full_text)} characters")
        
        # Save OCR output
        ocr_file = os.path.join(ERROR_LOG_DIR, f"{Path(pdf_path).stem}_ocr_text.txt")
        with open(ocr_file, 'w', encoding='utf-8') as f:
            f.write(full_text)
        
        return full_text
        
    except Exception as e:
        raise Exception(f"OCR failed: {str(e)}")

print("✓ OCR function defined")

In [ ]:
def process_pdf_with_fallback(pdf_path: str) -> str:
    """Try multiple extraction methods"""
    
    # Try pdfplumber first
    try:
        import pdfplumber
        print(f"    → Trying pdfplumber...")
        
        all_text = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    all_text.append(f"--- Page {i+1} ---\n{text}")
        
        full_text = "\n\n".join(all_text)
        
        if len(full_text) > 500:
            print(f"    ✓ pdfplumber: {len(full_text)} chars")
            return full_text
        else:
            print(f"    → Only {len(full_text)} chars, trying OCR...")
    except Exception as e:
        print(f"    → pdfplumber failed, trying OCR...")
    
    # Use OCR
    return process_scanned_pdf_with_ocr(pdf_path)

print("✓ PDF extraction with fallback defined")

In [ ]:
def extract_with_gemini(text_content: str, filename: str) -> dict:
    """Extract structured data using Google Gemini"""
    import google.generativeai as genai
    
    if not GEMINI_API_KEY:
        raise Exception("GEMINI_API_KEY not found")
    
    genai.configure(api_key=GEMINI_API_KEY)
    
    text_sample = text_content[:80000] if len(text_content) > 80000 else text_content
    
    print(f"    → Sending {len(text_sample)} chars to Gemini...")
    
    prompt = f"""Extract property valuation data from this Kenyan document (OCR text). Return ONLY valid JSON.

RULES:
- Extract EXACT values from document (no placeholders like "John Doe")
- If field not found, use "" or {{}}
- Amounts as numbers without commas (8500000 not "8,500,000")
- Dates as YYYY-MM-DD

Schema:
{{
  "property_id": "string",
  "valuation_report": {{
    "report_reference": "", "valuer": "", "valuers": [], "inspection_date": "",
    "report_date": "", "client": "", "client_address": "", "purpose": ""
  }},
  "property_details": {{
    "apartment_number": "", "block": "", "floor": "",
    "title_details": {{}}, "location": {{}}, "tenure": {{}},
    "registered_proprietors": [], "ownership_type": "", "encumbrances": ""
  }},
  "property_description": {{}},
  "apartment_details": {{}},
  "occupancy": "",
  "condition": "",
  "market_assessment": {{}},
  "valuations": {{
    "current_market_value": {{"amount": 0, "currency": "KES", "amount_words": ""}}
  }},
  "valuation_methodology": [],
  "lease_details": {{}},
  "compliance": {{}}
}}

Document:
{text_sample}

Return ONLY JSON:"""

    try:
        model = genai.GenerativeModel(GEMINI_MODEL)
        
        start_time = time.time()
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=4096,
            )
        )
        elapsed = time.time() - start_time
        
        response_text = response.text.strip()
        
        print(f"    → Gemini responded in {elapsed:.1f}s")
        
        # Save response
        debug_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_gemini_response.txt")
        with open(debug_file, 'w', encoding='utf-8') as f:
            f.write(response_text)
        
        # Clean JSON
        if '```json' in response_text:
            response_text = response_text.split('```json')[1].split('```')[0]
        elif '```' in response_text:
            response_text = response_text.split('```')[1].split('```')[0]
        
        response_text = response_text.strip()
        
        if not response_text.startswith('{'):
            start = response_text.find('{')
            end = response_text.rfind('}')
            if start != -1 and end != -1:
                response_text = response_text[start:end+1]
        
        extracted_data = json.loads(response_text)
        
        if not extracted_data.get('property_id'):
            extracted_data['property_id'] = Path(filename).stem
        
        return extracted_data
        
    except json.JSONDecodeError as e:
        raise Exception(f"JSON parsing failed: {str(e)}")
    except Exception as e:
        raise Exception(f"Gemini error: {str(e)}")

print("✓ Gemini extraction function defined")

In [ ]:
def process_single_pdf_complete(pdf_path: str, filename: str) -> dict:
    """Complete pipeline: OCR → Gemini → Validate → Save"""
    result = {
        'filename': filename,
        'success': False,
        'stage': None,
        'error': None,
        'data': None,
        'timing': {}
    }
    
    start_time = time.time()
    
    try:
        # Stage 1: OCR
        result['stage'] = 'ocr'
        print(f"\n  📄 Stage 1: OCR")
        stage_start = time.time()
        
        text_content = process_pdf_with_fallback(pdf_path)
        result['timing']['ocr'] = time.time() - stage_start
        
        if len(text_content) < 200:
            raise Exception(f"Insufficient text: {len(text_content)} chars")
        
        print(f"  ✓ {len(text_content)} chars in {result['timing']['ocr']:.1f}s")
        
        # Stage 2: Gemini
        result['stage'] = 'gemini'
        print(f"\n  🤖 Stage 2: Gemini")
        stage_start = time.time()
        
        extracted_data = extract_with_gemini(text_content, filename)
        result['timing']['gemini'] = time.time() - stage_start
        result['data'] = extracted_data
        
        print(f"  ✓ Extracted in {result['timing']['gemini']:.1f}s")
        
        # Stage 3: Validation
        result['stage'] = 'validation'
        print(f"\n  ✅ Stage 3: Validation")
        
        is_valid, validation_error = validate_extracted_data(extracted_data)
        if not is_valid:
            print(f"  ⚠ Warning: {validation_error[:100]}")
        else:
            print(f"  ✓ Valid")
        
        # Stage 4: Save
        result['stage'] = 'saving'
        print(f"\n  💾 Stage 4: Save")
        output_file = os.path.join(OUTPUT_DIR, f"{Path(filename).stem}.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, indent=2, ensure_ascii=False)
        print(f"  ✓ Saved to {Path(output_file).name}")
        
        # Stage 5: Supabase
        if supabase_client and is_valid:
            result['stage'] = 'supabase'
            try:
                upload_to_supabase(supabase_client, extracted_data, filename)
                print(f"  ✓ Uploaded to Supabase")
            except Exception as e:
                print(f"  ⚠ Supabase: {str(e)[:50]}")
        
        result['success'] = True
        result['timing']['total'] = time.time() - start_time
        
    except Exception as e:
        result['error'] = str(e)
        result['timing']['total'] = time.time() - start_time
        
        error_file = os.path.join(ERROR_LOG_DIR, f"{Path(filename).stem}_error.txt")
        with open(error_file, 'w', encoding='utf-8') as f:
            f.write(f"Stage: {result['stage']}\nError: {result['error']}\n")
        
        print(f"\n  ✗ Failed: {result['error'][:100]}")
    
    return result

print("✓ Complete pipeline defined")


In [ ]:
def process_all_pdfs_complete():
    """Process all PDFs"""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    
    if not pdf_files:
        print(f"No PDFs in {PDF_DIR}")
        return []
    
    print(f"\n{'='*70}")
    print(f"🚀 PROCESSING {len(pdf_files)} PDFs")
    print(f"{'='*70}")
    
    results = []
    total_start = time.time()
    
    for idx, pdf_file in enumerate(pdf_files, 1):
        print(f"\n{'='*70}")
        print(f"[{idx}/{len(pdf_files)}] {pdf_file.name}")
        print(f"{'='*70}")
        
        result = process_single_pdf_complete(str(pdf_file), pdf_file.name)
        results.append(result)
        
        if result['success']:
            print(f"\n✅ SUCCESS - {result['timing']['total']:.1f}s")
        else:
            print(f"\n❌ FAILED")
    
    # Summary
    total_time = time.time() - total_start
    successful = sum(1 for r in results if r['success'])
    
    print(f"\n{'='*70}")
    print(f"📊 SUMMARY")
    print(f"{'='*70}")
    print(f"Total: {len(results)} | Success: {successful} | Failed: {len(results)-successful}")
    print(f"Time: {total_time/60:.1f} min | Avg: {total_time/len(results):.1f}s/PDF")
    print(f"{'='*70}")
    
    # Save summary
    summary_file = os.path.join(OUTPUT_DIR, "summary.json")
    with open(summary_file, 'w') as f:
        json.dump({
            'timestamp': datetime.now().isoformat(),
            'model': GEMINI_MODEL,
            'total': len(results),
            'successful': successful,
            'results': results
        }, f, indent=2)
    
    return results

print("✓ Batch processing defined")

In [ ]:
def test_single_pdf():
    """Test with first PDF"""
    pdf_files = list(Path(PDF_DIR).glob("*.pdf"))
    if not pdf_files:
        print("No PDFs found")
        return
    
    test_file = pdf_files[0]
    print(f"\n🧪 TESTING: {test_file.name}\n")
    
    result = process_single_pdf_complete(str(test_file), test_file.name)
    
    if result['success']:
        print(f"\n✅ TEST PASSED!")
        print(f"\nPreview:")
        print(json.dumps(result['data'], indent=2)[:1000])
    else:
        print(f"\n❌ TEST FAILED")
    
    return result


print("\n" + "="*70)
print("✅ ALL FUNCTIONS LOADED")
print("="*70)
test_single_pdf()
# print("or:  results = process_all_pdfs_complete()")

✓ Test function defined

✅ ALL FUNCTIONS LOADED

Run: test_single_pdf()
or:  results = process_all_pdfs_complete()
